# 1 pysekiro 模块解析

## 1.1 按键部分
- get_keys.py (捕获键盘的按键)
- direct_keys.py (控制键盘的按键)
- actions.py (动作控制)

### 1.1.2 get_keys.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/get_keys.py)

**注：这部分是我从别人那里借鉴而来的**  
[源代码](https://github.com/Sentdex/pygta5/blob/master/getkeys.py)

检测按键的部分，用的是 virtual keys 。

~~~python
# virtual keys
vk = {
    'W' : 0x57,
    'S' : 0x53,
    'A' : 0x41,
    'D' : 0x44,

    'J' : 0x4A,
    'K' : 0x4B,
    'SPACE'    : 0x20,
    'LSHIFT'   : 0xA0,

    'T' : 0x54,
    'P' : 0x50
}
~~~

vk 是从 keys.py 中截取的部分键值对，里面的按键我已经定义好了，如果你想定义更多的按键，可以在 [keys.py](https://github.com/Sentdex/pygta5/blob/master/keys.py) 中第 146 ~ 254行找到相应的按键然后添加进去

然后，调用 key_check() 时就会检测当前正在按的键
~~~python
def key_check():
    keys = []
    for key in ['W', 'S', 'A', 'D', 'J', 'K', 'SPACE', 'LSHIFT', 'T', 'P']:
        if wapi.GetAsyncKeyState(vk[key]):
            keys.append(key)
    return keys
~~~

可以运行以下代码体验一下，按 P 停止

In [ ]:
from pysekiro.get_keys import key_check
while True:
    keys = key_check()
    print(f'\r{str(keys):<60}', end = '')
    if 'P' in keys:
        break

### 1.1.3 direct_keys.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/direct_keys.py)

**注：这部分是我从别人那里借鉴而来的**  
[源代码](https://github.com/Sentdex/pygta5/blob/master/directkeys.py)

PressKey 压键  
ReleaseKey 松键  

具体用法看 actions.py

### 1.1.4 actions.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/actions.py)

**注：这部分是我从别人那里借鉴而来的**  
[源代码](https://github.com/Sentdex/pygta5/blob/master/3.%20test_model.py)

输出按键的部分，用的是 direct keys 

#### 1.1.4.1 定义执行相应动作的函数

~~~python
# direct keys
dk = {
    'W' : 0x11,
    'S' : 0x1F,
    'A' : 0x1E,
    'D' : 0x20,

    'J' : 0x24,
    'K' : 0x25,
    'SPACE'    : 0x39,
    'LSHIFT'   : 0x2A,
}
~~~

dk 也是从 keys.py 中截取的部分键值对，里面的按键我已经定义好了，如果你想定义更多的按键，可以在 [keys.py](https://github.com/Sentdex/pygta5/blob/master/keys.py) 中第 35 ~ 143行找到相应的按键然后添加进去

动作定义
~~~python
# 先在 dk 里定义好相应的按键 
dk = {'W' : 0x11}

# 然后按照下面的示例定义执行相应动作的函数 
def Move_Forward():    # 移动 前
    PressKey(dk['W'])      # 压键
    time.sleep(1)    # 按键时长（控制长按短按）
    ReleaseKey(dk['W'])    # 松键
~~~

调用时，游戏就会接收到程序发送过来的按键信号，然后驱动人物做出相应的动作

#### 1.1.4.2 act(values)

[collect_data.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/collect_data.py) 中的 get_output() 影响的就是这里  
~~~python
"""
- 'J'      -> 0 -> Attack
- 'k'      -> 1 -> Deflect
- 'LSHIFT' -> 2 -> Step_Dodge
- 'SPACE'  -> 3 -> Jump
-          -> 4 -> 
"""
~~~

~~~python
# 根据 collect_data.py 中的 get_output()
def act(values):
    
    if   values == 0:
        act = Attack     # 攻击
    elif values == 1:
        act = Deflect    # 弹反
    elif values == 2:
        act = Step_Dodge # 垫步
    elif values == 3:
        act = Jump       # 跳跃
    elif values == 4:
        act = Move_Forward # 其他（暂时用 移动 前 替代）

    act_process = threading.Thread(target=act)
    act_process.start()
~~~

## 1.2 图像部分
- grab_screen.py (屏幕图像抓取)
- get_vertices.py (顶点位置获取)
- get_status.py (状态获取)

### 1.2.1 grab_screen.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/grab_screen.py)

**注：这部分是我从别人那里借鉴而来的**  
[源代码](https://github.com/Sentdex/pygta5/blob/master/grabscreen.py)

我修改了 grab_screen 的返回值
~~~python
return cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
~~~

追加了以下内容
~~~python
# 游戏窗口大小
GAME_WIDTH = 1280
GAME_HEIGHT = 720

# 标准窗口大小
STANDARD_WIDTH = 480
STANDARD_HEIGHT = 270

FRAME_COUNT = 1

def get_screen():
    # 屏幕捕获，并图像缩放
    screen = grab_screen(region=(0, 30, GAME_WIDTH, GAME_HEIGHT+30))
    screen = cv2.resize(screen, (STANDARD_WIDTH, STANDARD_HEIGHT))

    return screen
~~~

In [ ]:
import cv2
from pysekiro.grab_screen import get_screen

screen = get_screen()
cv2.imshow('screen', screen)
cv2.waitKey(0)

print('图像', screen)
print('形状', screen.shape)

### 1.2.2 get_vertices.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/get_vertices.py)

其实就是我无法写出一个自动把血条、架势条抠出来的代码，所以才写的这个半自动的程序。  
这个程序的主要作用是，在图像上点击然后自动返回相关的位置信息。这样就能获取到血条或者架势条在图像中的位置信息。  
以下是把人物抠出来的演示  
![demo.gif](https://github.com/ricagj/pysekiro_with_RL/blob/main/imgs/demo.gif?raw=true)

如图所示，在弹出来的窗口中用鼠标左键按顺序依次点击**左下**，**左上**，**右上**，**右下**，一共 4 次，然后按键盘上的“ESC”键，就会自动返回 x, x_w, y, y_h, vertices 。  
（注意：这个顺序是规定好的，可点击的次数也是规定好的）

其中    
x 代表 x轴的起始位置，也是图像的最左边  
y 代表 y轴的起始位置，也是图像的最上边  
x_w 代表 x轴的终止位置，也是图像的最右边  
y_h 代表 y轴的终止位置，也是图像的最下边  
vertices = [[x, y_h], [x, y], [x_w, y], [x_w, y_h]] (依次代表图像的左下角、左上角、右上角和右下角的位置。)  

**这些位置信息看起来有点奇怪，是因为图像的原点其实在左上角，想一下矩阵第一行或第一列在什么位置应该就理解了**

In [ ]:
# 参照着上面的演示自己试一下
import cv2
from pysekiro.get_vertices import get_vertices

img = cv2.imread(".\imgs\demo.png", 0)
get_vertices(img)

### 1.2.3 get_status.py

[查看代码](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/get_status.py)

![get_status](https://github.com/ricagj/pysekiro_with_RL/blob/main/imgs/adjustment.png?raw=true)

要获取的状态有：自身生命，自身架势，目标生命，目标架势。  
- 获取方法
    - 靠 [get_vertices.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/pysekiro/get_vertices.py) 把状态抠出来。
    - 用图像阈值处理（cv2.threshold）处理范围外的像素点（比如获取生命值状态，红色的部分是生命值，非红色的部分不是，那非红色的部分就是范围外），将其设为0
    - 通过计数法转化为数值

- 生命值的获取
    - 在灰度图中，像素点值的在一个固定的范围内，用图像阈值处理处理得好的话基本上能准确获取数值。

~~~python
# 获取自身生命
def get_Self_HP(img):
    img_roi = roi(img, x=29, x_w=182, y=246, y_h=246+1)[0]    # 获取自 get_vertices.py
    retval, img_th = cv2.threshold(img_roi, 60, 255, cv2.THRESH_TOZERO)    # 低于60的像素点的值设置为0
    retval, img_th = cv2.threshold(img_th, 80, 255, cv2.cv2.THRESH_TOZERO_INV)    # 高于80的像素点的值设置为0
    img_th = np.reshape(img_th, (img_roi.shape))
    Self_HP = get_value(img_th)    # 获取数值
    return Self_HP

# 获取目标生命  
# 代码和获取自身生命类似，这里就不贴出来了  
~~~

- 架势的获取
    - 架势和生命不同，它是从中间开始往两边增加的。我们没有必要两边都获取，只需要一边就够了。我选择的是中间和右边。
    - 还有几个问题比较麻烦，就是架势在归零的时候会不显示，积累到一定时候颜色会变化，生命值低还有异常状态的时候也和正常的时候不一样。设置开启条件后能较稳定的获取数值。
    - 所谓的开启条件，就是架势条中间那条白色的线所在的像素点以及它右边第一个像素点，当这两个像素点满足一定条件，才记录架势。

~~~python
# 获取自身架势
def get_Self_Posture(img):
    img_roi = roi(img, x=240, x_w=290, y=234, y_h=234+1)[0]    # 获取自 get_vertices.py
    retval, img_th = cv2.threshold(img_roi, 100, 255, cv2.THRESH_TOZERO)    # 低于100的像素点的值设置为0
    img_th = np.reshape(img_th, (img_roi.shape))
    if int(img_th[0]) - int(img_th[1]) > 15:    # 开启条件
        if img_th[1] in range(100, 125) and img_th[0] in range(145, 165):
            Self_Posture = get_value(img_th)
        elif img_th[1] in range(135, 160) and img_th[0] in range(180, 220):
            Self_Posture = get_value(img_th)
        elif img_th[1] in range(160, 230) and img_th[0] in range(200, 250):
            Self_Posture = get_value(img_th)
        else:
            Self_Posture = 0
    else:
        Self_Posture = 0
    return Self_Posture

# 获取目标架势
# 代码和获取自身架势类似，这里就不贴出来了
~~~

获取数值  
范围外的像素点已经通过图像阈值处理处理成0了，所以计算第一个出现的0的左边有多少个非0值，得到的结果就是我们需要的那个状态值
~~~python
def get_value(target_img):
    count = 0
    for i in range(len(target_img)-1):
        cur_pixel = int(target_img[i])
        if cur_pixel == 0:
            break
        count += 1
    return count
~~~

不理解的话就把 get_status.py 内的以下代码取消注释然后去运行下面的代码
~~~python
#     print('\n', img_th)
~~~

~~~python
# 运行这个
from pysekiro.collect_data import Data_collection
target = 'Genichiro_Ashina' # 苇名弦一郎
c = Data_collection(target)
c.collect_data()
~~~